## Serializing and Deserializing Strings

In [31]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import StringSerializer
import json
from uuid import uuid4

In [46]:
topic = "topic1"
brokers = "kafka1:9092,kafka2:9093,"

### Producer

Reports the failure or success of a message delivery.
Args:  
    - err (KafkaError): The error that occurred on None on success.
    - msg (Message): The message that was produced or failed.

Note: In the delivery report callback the Message.key() and Message.value() will be the binary format as encoded by any configured Serializers and not the same object that was passed to produce().
If you wish to pass the original object(s) for key and value to delivery
report callback we recommend a bound callback or lambda where you pass  the objects along.

In [47]:
def delivery_report(err, msg):

    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [48]:
producer_conf = {
        'bootstrap.servers': brokers,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')}

In [49]:
producer = SerializingProducer(producer_conf)

In [63]:
with open('./data.json') as f:
    data = json.load(f)

In [65]:
for m in data:
    producer.poll(0.0)
    producer.produce(topic=topic, key=str(uuid4()), 
                     value=str(m),
                     on_delivery=delivery_report)
p.poll(0)

User record b'91bf99a8-4d4c-4e38-94ed-71f32849a81b' successfully produced to topic1 [3] at offset 2
User record b'fe8f0ab3-172c-47b9-8e7d-be6c086b1400' successfully produced to topic1 [3] at offset 3
User record b'bec0fac6-e9e5-46a9-aa69-09acf5c68d84' successfully produced to topic1 [4] at offset 2
User record b'f4efbd1e-e268-4a42-a48f-6ad609fb72e4' successfully produced to topic1 [4] at offset 3
User record b'69936374-5fa1-46b7-b78e-5022128a7a8f' successfully produced to topic1 [0] at offset 1


% Message delivered to test [0] @ 6
% Message delivered to test [0] @ 7
% Message delivered to test [0] @ 8
% Message delivered to test [0] @ 9
% Message delivered to test [0] @ 10
% Message delivered to test [0] @ 11
% Message delivered to test [0] @ 12
% Message delivered to test [0] @ 13
% Message delivered to test [0] @ 14


1

In [66]:
producer.flush()

User record b'5069b8ee-263c-4d77-bb0b-549102db9ccc' successfully produced to topic1 [3] at offset 4
User record b'f674fb13-1616-4212-bdfe-a8b758458305' successfully produced to topic1 [0] at offset 2
User record b'64d04480-36c5-4a3e-9653-0ca387026787' successfully produced to topic1 [2] at offset 1
User record b'2efd1631-bedf-40a7-af0d-f6cc8be3a07b' successfully produced to topic1 [2] at offset 2
User record b'7bc38672-a918-4c97-8a9e-dd79d112b5f1' successfully produced to topic1 [2] at offset 3


0

### Consumer

In [67]:
from confluent_kafka import DeserializingConsumer
from confluent_kafka.serialization import StringDeserializer

In [68]:
string_deserializer = StringDeserializer('utf_8')

In [69]:
consumer_conf = {'bootstrap.servers': brokers,
                     'key.deserializer': string_deserializer,
                     'value.deserializer': string_deserializer,
                     'group.id': "consumer5"}

In [70]:
consumer = DeserializingConsumer(consumer_conf)
consumer.subscribe([topic])

In [ ]:
while True:
    try:
        # SIGINT can't be handled when polling, limit timeout to 1 second.
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        m = msg.value()
        if m is not None:
            print(m)
    except KeyboardInterrupt:
        break

consumer.close()

## Avro